In [230]:
import numpy as np
import svgwrite
from IPython.display import SVG
from IPython.display import HTML




In [6]:
blank = 0
vcar = 4
vtruck = 5
hcar = 6
htruck = 7

CAR_SYMBOLS = ['Q', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L']
TRUCK_SYMBOLS = ['T', 'R', 'W', 'Z']

CAR_COLORS = ['#7FFF00', '#7FFFD4', '#D2691E', '#8B008B', '#BDB76B'\
              '#8B0000', '#FF1493', '#1E90FF', '#FFD700', '#ADFF2F',\
              '#CD5C5C', '#F0E68C']

TRUCK_COLORS = ['#F08080', '#FFA07A', '#FF00FF', '#00FA9A']

RED_COLOR = '#FF0000'
RED_SYMBOL = 'X'


Create a baseline svg image upon which specific car layouts will be drawn

In [216]:
space_size = 30
board_size = 6*space_size

def svg_base():
    dwg = svgwrite.Drawing('nosave.svg',(board_size,board_size),debug=True)

    dwg.add(dwg.rect(insert=(0,0),size=(board_size,board_size),fill='#E6E6E6'))
    #dwg.add(dwg.rect(insert=(0,0),size=(board_size,board_size),fill='rgb(211,211,211)'))
    for x in range(7):
        dwg.add(dwg.line((30*x,0),(30*x,180),stroke='black',stroke_width=2))
        dwg.add(dwg.line((0,30*x),(180,30*x),stroke='black',stroke_width=2))
    return dwg

# TEST
dwg = svg_base()
HTML(dwg.tostring())


In [217]:
# orientation values: 'vcar',hcar',vtruck','htruck'
# color is hex value color (e.g. #7FFFD4)
space_size = 30
board_size = 6 * space_size
svg_border = 3 
svg_round_radius = 5

svg_size = {}
svg_size['hcar'] = (2*space_size - 2*svg_border , space_size - 2*svg_border)
svg_size['vcar'] = (space_size - 2*svg_border,  2*space_size - 2*svg_border)
svg_size['vtruck'] = (space_size - 2*svg_border , 3*space_size - 2*svg_border)
svg_size['htruck'] = (3*space_size - 2*svg_border, space_size - 2*svg_border)

#dwg.add(dwg.rect(insert=(65, 35), size=(50, 20),rx=5,ry=5,fill='green',  stroke_width=3))
def svg_add_piece(dwg,nd_x,nd_y,orientation,color,text):
    car_x = nd_y* space_size + svg_border
    car_y = nd_x*space_size + svg_border
    
    size = svg_size[orientation]
    
    dwg.add(dwg.rect(insert=(car_x,car_y),size=size,rx=svg_round_radius,ry=svg_round_radius,fill=color, stroke_width=3))
    
    c_x = car_x + size[0]/2.0
    c_y = car_y + size[1]/2.0
    dwg.add(dwg.text(text,insert=(c_x,c_y),style='fill:black;text-anchor:middle;alignment-baseline:central'))
    return dwg


# TEST
dwg = svg_base()
dwg = svg_add_piece(dwg, 0,0,'vcar','blue','P')
dwg = svg_add_piece(dwg, 1,1,'hcar','pink','Q')
dwg = svg_add_piece(dwg, 2,2,'vtruck','green','R')
dwg = svg_add_piece(dwg, 3,3,'htruck','yellow','T')




HTML(dwg.tostring())
        

In [14]:
def svg_from_state(t,red_cols):
    ''' 
        input t: numpy array modeling a RH board configuration
        input red_cols: 2-element array [c1,c2] that marks the columns of the red car 

        output: instance of svgwrite drawing
        
    '''
    
    v = np.copy(t)
    c1,c2 = red_cols
    
    car_colors = CAR_COLORS[:]
    truck_colors = TRUCK_COLORS[:]
    
    truck_symbols = TRUCK_SYMBOLS[:]
    car_symbols = CAR_SYMBOLS[:]
    
    dwg = svg_base()
    
    # zero out red car. We explicitly know how to mark it visually
    v[2,c1] = blank
    v[2,c2] = blank
    dwg = svg_add_piece(dwg,2,c1,'hcar',RED_COLOR,RED_SYMBOL)
    
    vcars_y =np.unique(np.where(v==vcar)[1])
    for y in vcars_y:
        for x in np.where(v[:,y] == vcar)[0][0::2]:          
            dwg = svg_add_piece(dwg,int(x),int(y),'vcar',car_colors.pop(),car_symbols.pop())
    
    hcars_x = np.unique(np.where(v==hcar)[0])
    for x in hcars_x:
        for y in np.where(v[x,:]==hcar)[0][0::2]:
            dwg = svg_add_piece(dwg,int(x),int(y),'hcar',car_colors.pop(),car_symbols.pop())
    
    vtrucks_y = np.unique(np.where(v==vtruck)[1])
    for y in vtrucks_y:
        for x in np.where(v[:,y] == vtruck)[0][0::3]:
            dwg = svg_add_piece(dwg,int(x),int(y),'vtruck',truck_colors.pop(),truck_symbols.pop())
    
    htrucks_x = np.unique(np.where(v==htruck)[0])
    for x in htrucks_x:
        for y in np.where(v[x,:]==htruck)[0][0::3]:
            dwg = svg_add_piece(dwg,int(x),int(y),'htruck',truck_colors.pop(),truck_symbols.pop())
    
    return dwg

In [19]:
# TEST

t = np.zeros((6,6),dtype=int)
t[2,0:2] = hcar
t[1,4:6] = hcar
t[2:4,4] = vcar

#t[0,2:5] = htruck
t[0,1:4] = htruck

t[3:6,2] = vtruck
t[4:,0] = vcar
t

array([[0, 7, 7, 7, 0, 0],
       [0, 0, 0, 0, 6, 6],
       [6, 6, 0, 0, 4, 0],
       [0, 0, 5, 0, 4, 0],
       [4, 0, 5, 0, 0, 0],
       [4, 0, 5, 0, 0, 0]])

In [20]:
dwg = svg_from_state(t,[0,1])
HTML(dwg.tostring())

In [246]:
def svg_from_state_generated_order(v,red_cols):
    
    v = np.copy(t)

    c1,c2 = red_cols

    car_colors = CAR_COLORS[:]
    truck_colors = TRUCK_COLORS[:]

    truck_symbols = TRUCK_SYMBOLS[:]
    car_symbols = CAR_SYMBOLS[:]

    dwg = svg_base();

    # zero out red car. We explicitly know how to mark it visually
    v[2,c1] = blank
    v[2,c2] = blank
    dwg = svg_add_piece(dwg,2,c1,'hcar',RED_COLOR,RED_SYMBOL)

    end_positions = []
    
    vcars_y =np.unique(np.where(v==vcar)[1])
    for y in vcars_y:
        for x in np.where(v[:,y] == vcar)[0][0::2]:          
            #dwg = svg_add_piece(dwg,int(x),int(y),'vcar',car_colors.pop(),car_symbols.pop())
            end_positions.append([int(x),int(y)])

    hcars_x = np.unique(np.where(v==hcar)[0])
    for x in hcars_x:
        for y in np.where(v[x,:]==hcar)[0][0::2]:
            #dwg = svg_add_piece(dwg,int(x),int(y),'hcar',car_colors.pop(),car_symbols.pop())
            end_positions.append([int(x),int(y)])

    vtrucks_y = np.unique(np.where(v==vtruck)[1])
    for y in vtrucks_y:
        for x in np.where(v[:,y] == vtruck)[0][0::3]:
            #dwg = svg_add_piece(dwg,int(x),int(y),'vtruck',truck_colors.pop(),truck_symbols.pop())
            end_positions.append([int(x),int(y)])

    htrucks_x = np.unique(np.where(v==htruck)[0])
    for x in htrucks_x:
        for y in np.where(v[x,:]==htruck)[0][0::3]:
            #dwg = svg_add_piece(dwg,int(x),int(y),'htruck',truck_colors.pop(),truck_symbols.pop())
            end_positions.append([int(x),int(y)])

    end_positions = sorted(end_positions)
    for row,col in end_positions:
        if v[row,col] == hcar:
            dwg = svg_add_piece(dwg,int(row),int(col),'hcar',car_colors.pop(),car_symbols.pop() )
        elif v[row,col] == vcar:
            dwg = svg_add_piece(dwg,int(row),int(col),'vcar',car_colors.pop(),car_symbols.pop() )
        elif v[row,col] == htruck:
            dwg = svg_add_piece(dwg,int(row),int(col),'htruck',truck_colors.pop(),truck_symbols.pop() )
        else:
            dwg = svg_add_piece(dwg,int(row),int(col),'vtruck',truck_colors.pop(),truck_symbols.pop() )
    return dwg


In [247]:
HTML(svg_from_state_generated_order(t,[0,1]).tostring())

In [241]:
sorted(end_positions)

[[0, 1], [1, 4], [2, 4], [3, 2], [4, 0]]